In [1]:
import sys
import os

# Add the parent directory of 'vegetation-period-NDVI' to the sys.path
sys.path.append(os.path.abspath(os.path.join('..')))


In [2]:
import ee
import geemap
from time_series import extract_time_ranges, get_harmonic_ts
from vegetation_period_extraction import get_crop_veg_period, create_binary_ndvi_indicator, create_binary_mask

In [3]:
ee.Initialize(project="thurgau-irrigation")

In [4]:
aoi = ee.Geometry.Polygon(
    [
        [
            [8.737412756413452, 47.648802254332814],
            [8.737412756413452, 47.61548626028342],
            [8.829423254460327, 47.61548626028342],
            [8.829423254460327, 47.648802254332814],
        ]
    ]
)

In [5]:
year = 2018

# Define the time range for analysis
start_date = f"{year}-03-01"
end_date = f"{year}-10-31"

# Create time intervals
time_intervals = extract_time_ranges([start_date, end_date], 15)

# Get the vegetation period estimation
veg_period_image = get_crop_veg_period(year, aoi, time_intervals)

In [6]:
type(veg_period_image)

ee.image.Image

In [12]:
double_band = veg_period_image.select('double')

# Create a map
Map = geemap.Map(zoom=10)
Map.centerObject(aoi)
# Add the 'double' band to the map
Map.addLayer(double_band, {'min': 0, 'max': 1, 'palette': ['blue', 'green', 'red']}, 'Double Band')

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
filtered_sentinel_data = get_harmonic_ts(year=year, aoi=aoi, time_intervals=time_intervals)

In [9]:
filtered_sentinel_data

{'fitted_data': <ee.imagecollection.ImageCollection at 0x15fcf8190>,
 'regression_coefficients': <ee.image.Image at 0x15f8eb590>,
 'phase_amplitude': <ee.image.Image at 0x15fc5ca90>}

In [14]:
Map = geemap.Map()


# Add the layer to the map.
ndvi_image = ee.Image(filtered_sentinel_data.get("fitted_data").toList(99).get(7))
Map.centerObject(aoi, 13)
ndvi_params = {'bands': ['rmse'], 'min': 0, 'max': 1, 'palette': ['white', 'green']}
Map.addLayer(ndvi_image, ndvi_params, 'NDVI FIRST', True)

# Display the map.
Map

Map(center=[47.632151683310106, 8.783418005435093], controls=(WidgetControl(options=['position', 'transparent_…